In [40]:
def ppc(sg):
    
    '''This funtion is for calculate the psedo pressure compresibility (ppc) 
    with the correlation develop by sutton,receive as input specific gravity
    and as output the ppc in psia.The correlation work with a value of sg 
    between 0.57 and 1.68'''

    if 0.57<=sg<=1.68:
        ppc=756.8-(131*sg)-(3.6*sg*sg)    
    else:
        ppc=print("The ppc correlation work with sg values between 0.57 and 1.68")
    return ppc



In [45]:
def ppr(sg,p):

    '''This funtion calculate the pseudocritical pressure (ppr)
    as input need the especific gravity (adimensional) and pressure in psi 
    and as output the ppr (adimensional) '''
    
    return p/ppc(sg)

In [32]:
def tpc(sg):
    
    '''This funtion is for calculate the psedo temperature compresibility (tpc)
    with the correlation develop by sutton, receive as input specific gravity
    and as output the tpc in Rankin.The correlation work with a value of sg 
    between 0.57 and 1.68'''

    if 0.57<=sg<=1.68:
        tpc=169.2+(349.5*sg)-(74*sg*sg) 
    else:
        tpc=print("The tpc correlation work with sg values between 0.57 and 1.69")
    return tpc

In [49]:
def tpr(sg,t):

    '''This funtion calculathe the pseudocritical temperature (tpr)
    as input need the specific gravity (adimensinal) and temperature in farenheit
    and as aouput receive the tpr (adimensinal)'''

    return (460+t)/tpc(sg)

In [50]:
print(tpc(0.665))

368.89285


In [51]:
print(tpr(0.665,213))

1.8243780002784005
